In [30]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [31]:
# raw_data = pd.ExcelFile('rbull1.xlsx')

In [32]:
# raw_data.sheet_names

In [33]:
# data = raw_data.parse('rbull1.csv')

In [34]:
cols =['DateStart','AdsetId','LinkClick','Conversions','Impressions']
data = pd.read_csv('RagingBull_FB_Hourly.csv', usecols=cols)
data.to_csv('Final_Set')
data.fillna(0,inplace=True)

In [35]:
data['CTR'] = (data['LinkClick']/data['Impressions']).replace([np.inf, -np.inf], 0)

In [36]:
data.head()

,DateStart,LinkClick,Impressions,AdsetId,Conversions,CTR
0,2018-07-12,0.0,27.0,6094650899637,0.0,0.0
1,2018-05-18,0.0,3.0,6093425908037,0.0,0.0
2,2018-05-26,0.0,2.0,6093224252437,0.0,0.0
3,2018-06-07,0.0,7.0,6094650898437,0.0,0.0
4,2018-06-04,0.0,2.0,6095173565837,0.0,0.0


In [37]:
data_total_CTR = pd.pivot_table(data = data, values= ['LinkClick','Impressions'], fill_value= 0, index='AdsetId',aggfunc='sum')

In [38]:
data_total_CTR['CTR_agg'] = (data_total_CTR['LinkClick']/data_total_CTR['Impressions']).replace([np.inf, -np.inf], 0)
data_total_CTR.drop(['LinkClick','Impressions'],inplace = True, axis = 1)
data_first_row = pd.pivot_table(data = data_total_CTR, values= ['CTR_agg'],columns='AdsetId').replace([np.inf, -np.inf], 0)

In [39]:
data_first_row.head()

AdsetId,6082903610237,6082903612037,6082903614437,6082903615637,6082903618837,6082903621237,6088935250037,6088935251437,6088935252837,6089948390437,...,23842904874210027,23842974432060616,23842974432080616,23842974432090616,23842974432100616,23842974432160616,23842974432170616,23843001848720616,23843001951860616,23843001969560616
CTR_agg,0.013802,0.01367,0.010551,0.009154,0.012005,0.013056,0.009543,0.011484,0.010145,0.01272,...,0.013622,0.008832,0.012568,0.014083,0.0051,0.009281,0.009237,0.005055,0.004031,0.005489


In [40]:
data_total_CTR.head()

,CTR_agg
AdsetId,
6082903610237,0.013802
6082903612037,0.013670
6082903614437,0.010551
6082903615637,0.009154
6082903618837,0.012005


In [41]:
data = data.join(data_total_CTR, on = ['AdsetId'])

In [42]:
data = data[data['CTR_agg'] > 0]

In [43]:
data.drop(['CTR_agg'], inplace= True, axis = 1)

In [44]:
data.head()

,DateStart,LinkClick,Impressions,AdsetId,Conversions,CTR
0,2018-07-12,0.0,27.0,6094650899637,0.0,0.0
1,2018-05-18,0.0,3.0,6093425908037,0.0,0.0
2,2018-05-26,0.0,2.0,6093224252437,0.0,0.0
3,2018-06-07,0.0,7.0,6094650898437,0.0,0.0
4,2018-06-04,0.0,2.0,6095173565837,0.0,0.0


In [45]:
data.shape

(438872, 6)

In [46]:
len(set(data['AdsetId']))

242

In [47]:
opt_data = (data[['DateStart','AdsetId','CTR']].copy(deep = True))

In [48]:
data_final = pd.pivot_table(data=opt_data, values= 'CTR', columns='AdsetId',fill_value= 0,index='DateStart',aggfunc='sum').reset_index()
data_final.drop('DateStart',inplace= True, axis=1)
data_final.shape

(130, 242)

In [49]:
# data_initial_set = pd.pivot_table(data = data, values= ['LinkClick','conversions'], fill_value= 0, index='id',aggfunc='sum')

In [50]:
# data_initial_set['TotalClicksperConversion'] = (data_initial_set['LinkClick']/data_initial_set['Conversions']).replace([np.inf, -np.inf,np.NAN], 0)

In [51]:
# data_initial_set = pd.pivot_table(data = data_initial_set, columns= 'id', values= 'TotalClicksperConversion', fill_value= 0).reset_index()

In [52]:
# data_initial_set.drop('index',inplace= True,axis =1)

In [53]:
# data_initial_set

In [54]:
# import gc
# del opt_data
# # del raw_data
# gc.collect()

In [55]:
len_data_final = data_final.shape[1]
df = pd.DataFrame()
appender = data_first_row.iloc[[0]]
while len_data_final >= 0:
    df = df.append(appender,ignore_index= True)
    len_data_final -= 1
df.shape 

(243, 242)

In [56]:
data_final = df.append(data_final)
data_final.shape

(373, 242)

In [81]:
import random
import math
from collections import defaultdict
reward = 0
N = data_final.shape[0]
d = data_final.shape[1]
Exp = 5 ###Number of experiments
experiment_data = defaultdict()
for exp in range(Exp):
    id_reward_map = defaultdict()
    id_reward_map = {}
    numbers_of_rewards_1 = [0] * d
    numbers_of_rewards_0 = [0] * d
    sums_of_rewards = [0] * d
    total_reward = 0

    for key, value in enumerate(data_final.columns):
        id_reward_map[key] = [0, value, 0]

    for n in range(0, N):
        ad = 0
        max_random = 0
        for i in range(0, d):
            random_beta = random.betavariate(numbers_of_rewards_1[i] + 1, numbers_of_rewards_0[i] + 1)
    #         print("Ad id {}  Max random .{}  Random beta .{}".format(i, max_random, random_beta))
    #         print("".format())
            if random_beta > max_random:
                max_random = random_beta
                ad = i
    #             print(ad)
                id_reward_map[ad][0] += 1
        #     print(ads_selected)
                reward = data_final.values[n, ad]
                if reward > 0:
                    numbers_of_rewards_1[ad]+= 1
                    id_reward_map[ad][2]+=reward
                    sums_of_rewards[ad] = sums_of_rewards[ad] + reward
                    total_reward += reward
                else:
                    numbers_of_rewards_0[ad]+= 1
#             print("Reward {}".format(reward))
#             print("Total re")
                
    experiment_data[exp] = {'id_reward_map': id_reward_map, 'total_reward': total_reward}

    #Visualising the results
    # plt.hist(id_reward_map[1])
    # plt.title('Histogram of ads selections')
    # plt.xlabel('Ads')
    # plt.ylabel('Number of times each ad was selected')
    # plt.show()

In [82]:
val = sorted([(value['total_reward'],key) for (key,value) in experiment_data.items()], reverse=True)[:1]
val

[(255.99764996033838, 4)]

In [83]:
total_weights = val[0][0]
experiment_selected = val[0][1]
# Weights_selected = 
ad_weights = defaultdict()
for key, value in experiment_data.items():
    for k,val in value['id_reward_map'].items():
        ad_weights[val[1]] = val[2]

In [84]:
sum(ad_weights.values())

255.9976499603385

In [85]:
print(sum(sums_of_rewards))

255.9976499603385


In [86]:
total_reward

255.99764996033838